In [2]:
# Packages
import pandas as pd
from scipy.io import arff
import pandas as pd

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 3: Unsupervised learning with PCA, t-SNE, k-means, AHC and SOM

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez

Part 1: Selecting and analyzing the datasets

The unsupervised learning techniques must be applied on two datasets:

In [10]:
# We upload the datasets

# First dataset: File: A3-data.txt
    # Features: 4 variables, 1 class
    # Patterns: 360 patterns
df_data=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A3/A3-data.txt', sep=',', header=None)
header_vector_data = df_data.iloc[0, :].tolist() #header
df_data=df_data.iloc[1:,:-1]
df_data=pd.DataFrame(df_data)

# Second dataset: from "https://www.openml.org/search?type=data&status=active&id=188"
    # Features: at least 6 variables, and a class attribute
    # The class attribute must refer to, at least, 4 different classes
    #  Patterns: at least 200 patterns
data_Eucaliptus, meta = arff.loadarff('C:/Users/Gari/Desktop/Assignments_NEC/A3/dataset_194_eucalyptus.arff')
# Convert numpy array to DataFrame
df_Eucaliptus = pd.DataFrame(data_Eucaliptus)